In [4]:
from spektral.layers import GCNConv

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import tensorflow as tf
from tensorflow.keras.regularizers import l2
import pandas as pd

In [5]:
import numpy as np
import scipy.sparse as sp
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from spektral.data import Dataset, DisjointLoader, Graph
from spektral.layers import GCSConv, GlobalAvgPool, ChebConv
from spektral.layers.pooling import TopKPool
from spektral.transforms.normalize_adj import NormalizeAdj
import pickle as pkl
import networkx as nx
import pdb
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [53]:
################################################################################
# PARAMETERS
################################################################################
learning_rate = 1e-2  # Learning rate
epochs = 1000  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 1  # Batch size


################################################################################
# LOAD DATA
################################################################################
import sys
from spektral.data import Dataset, Graph
# sys.path.append('../lib')
# from data_pre_processing import load_data
# sys.path.remove('../lib')
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import AdjToSpTensor, LayerPreprocess

def load_data(circuit_name, path_to_data="data", normalize=False):
    """Load data."""
    names = ["x", "y", "graph"]
    objects = []
    for i in range(len(names)):
        with open(f"{path_to_data}/{circuit_name}.{names[i]}", "rb") as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding="latin1"))
            else:
                objects.append(pkl.load(f))

    x, y, graph = tuple(objects)

    features = _preprocess_features(sp.csr_matrix(x).astype('float32'))
    # adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph)).astype(int)
    g = nx.DiGraph()
    g.add_nodes_from(graph.keys())
    for k, v in graph.items():
      g.add_edges_from(([(k, t) for t in v]))
      g.add_edges_from([(k, k)])
    adj = nx.adjacency_matrix(g)
    labels = np.array(y).astype('float32').reshape((-1,1))

    print(adj.shape)
    print(features.shape)
    return adj, features, labels
  
def encode_label(labels):
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    labels = to_categorical(labels)
    return labels


def _preprocess_features(features):
    rowsum = np.array(features.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.0
    r_mat_inv = sp.diags(r_inv)
    features = r_mat_inv.dot(features)
    return features

class CircuitDataset(Dataset):
    def read(self):
        circuits = []
        circs = ['c6288','c5315','c432', 'c499', 'c880', 'c1355', 'c1908', 'c3540', 'adder.bench', 'arbiter.bench', 'cavlc.bench', 'dec.bench', 'voter.bench']
        # circs = ['c17']
        for circ in circs:
            A, X, labels = load_data(circ, '../data/output', normalize="")
            circuits.append(Graph(x=X.toarray(), a=A, y=labels))
            print(f"{circ}: {sum(labels)}, {len(labels)}")
        return circuits

dataset = CircuitDataset(transforms=[LayerPreprocess(ChebConv)])

# Parameters
F = dataset.n_node_features  # Dimension of node features
n_out = dataset.n_labels  # Dimension of the target

# Train/valid/test split
idxs = np.random.permutation(len(dataset))
split_va, split_te = int(0.8 * len(dataset)), int(0.9 * len(dataset))
idx_tr, idx_va, idx_te = np.split(idxs, [split_va, split_te])
print(idx_tr, idx_va, idx_te)
dataset_tr = dataset[idx_tr]
dataset_va = dataset[idx_va]
dataset_te = dataset[idx_te]
# dataset_tr = dataset[[8,6,5,1,2, 0, 3, 4, 7]]
# dataset_va = dataset[[0,3]]
# dataset_te = dataset[4,7]
# loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs, node_level=True)
# loader_va = DisjointLoader(dataset_va, batch_size=batch_size, node_level=True)
# loader_te = DisjointLoader(dataset_te, batch_size=batch_size, node_level=True)
def load_tr_data(epochs=400):
    loader_tr = DisjointLoader(dataset_tr, batch_size=batch_size, epochs=epochs, node_level=True)
    loader_va = DisjointLoader(dataset_va, batch_size=batch_size, node_level=True)
    loader_te = DisjointLoader(dataset_te, batch_size=batch_size, node_level=True)
    return loader_tr, loader_va, loader_te

loader_tr, loader_va, loader_te = load_tr_data()


(6288, 6288)
(6288, 4)
c6288: [846.], 6288
(5315, 5315)
(5315, 4)
c5315: [599.], 5315
(432, 432)
(432, 4)
c432: [60.], 432
(499, 499)
(499, 4)
c499: [50.], 499
(880, 880)
(880, 4)
c880: [114.], 880
(1355, 1355)
(1355, 4)
c1355: [192.], 1355
(1908, 1908)
(1908, 4)
c1908: [257.], 1908
(3540, 3540)
(3540, 4)
c3540: [406.], 3540
(4209, 4209)
(4209, 4)
adder.bench: [637.], 4209
(36352, 36352)
(36352, 4)
arbiter.bench: [3763.], 36352
(2164, 2164)
(2164, 4)
cavlc.bench: [318.], 2164
(936, 936)
(936, 4)
dec.bench: [186.], 936
(50122, 50122)
(50122, 4)
voter.bench: [5012.], 50122
[ 8  7  3  9 11  1  6  5  0 12] [2] [10  4]


In [54]:
N = sum([ g.n_nodes for g in dataset.graphs ])
n_out = 1
N

114000

In [55]:
dataset_tr.n_node_features

4

In [56]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [57]:
from tensorflow.keras.metrics import sparse_categorical_accuracy, binary_accuracy, binary_crossentropy
channels = 256         # Number of channels in the first layer
dropout = 0.05         # Dropout rate for the features
l2_reg = 5e-4           # L2 regularization rate
learning_rate = 1e-2    # Learning rate
epochs = 1000            # Number of training epochs
es_patience = 10        # Patience for early stopping
# N = 

# Model definition
X_in = Input(shape=(F, ))
print("X_in: ", X_in.shape)
fltr_in = Input((None, ), sparse=True)
mask_in = Input((None, ))

x_1 = BatchNormalization()(X_in)
dropout_1 = Dropout(dropout)(x_1)
graph_conv_1 = ChebConv(512, K=2,
                         activation='relu',
                         use_bias=True, n_input_channels=dataset.n_node_features)([dropout_1, fltr_in])
x_2 = BatchNormalization()(graph_conv_1)
dropout_2 = Dropout(dropout)(x_2)
graph_conv_1 = ChebConv(128, K=2,
                         activation='relu',
                         use_bias=True, n_input_channels=dataset.n_node_features)([dropout_2, fltr_in])
x_2 = BatchNormalization()(graph_conv_1)
dropout_2 = Dropout(dropout)(x_2)
graph_conv_1 = ChebConv(64, K=2,
                         activation='relu',
                         use_bias=True, n_input_channels=dataset.n_node_features)([dropout_2, fltr_in])
x_2 = BatchNormalization()(graph_conv_1)
dropout_2 = Dropout(dropout)(x_2)
graph_conv_3 = GCNConv(n_out,
                         activation='sigmoid',
                         use_bias=True, n_input_channels=dataset.n_node_features)([dropout_2, fltr_in])

model = Model(inputs=[X_in, fltr_in, mask_in], outputs=graph_conv_3)
optimizer = Adam(lr=learning_rate)

def masked_loss_and_accuracy(target, predictions):
    loss = binary_crossentropy(target, predictions)
    minimum = tf.math.minimum(tf.reduce_sum(tf.dtypes.cast(target == 0, tf.int32)), tf.reduce_sum(tf.dtypes.cast(target == 1, tf.int32)))
    zeroes_index = tf.random.shuffle(tf.where(target == 0)[:,0])
    ones_index = tf.random.shuffle(tf.where(target == 1)[:,0])
    loss_zeroes =  tf.reduce_mean(tf.gather(loss, zeroes_index))
    loss_ones = tf.reduce_mean(tf.gather(loss, ones_index))
    
    weight_for_0 = tf.cast((1 / tf.shape(zeroes_index)[0])*(tf.cast(tf.shape(target)[0], 'float64'))/2.0, 'float32')
    weight_for_1 = tf.cast((1 / tf.shape(ones_index)[0])*(tf.cast(tf.shape(target)[0], 'float64'))/2.0, 'float32')

    return (weight_for_0 * loss_zeroes * 0.6 +  weight_for_1 * loss_ones * 0.4)


model.compile(optimizer=optimizer,
              loss=masked_loss_and_accuracy,
              weighted_metrics=['acc', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.summary()

tbCallBack_GCN = tf.keras.callbacks.TensorBoard(
        log_dir='./DetectFaultInCircuit12',
        histogram_freq=1,
        write_graph=True,
        write_images=True
)
callback_GCN = [tbCallBack_GCN]

X_in:  (None, 4)
Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_43 (InputLayer)           [(None, 4)]          0                                            
__________________________________________________________________________________________________
batch_normalization_40 (BatchNo (None, 4)            16          input_43[0][0]                   
__________________________________________________________________________________________________
dropout_39 (Dropout)            (None, 4)            0           batch_normalization_40[0][0]     
__________________________________________________________________________________________________
input_44 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________

In [59]:
rm -rf ./DetectFaultInCircuit12

In [60]:
loader_tr, loader_va, loader_te = load_tr_data(epochs)
history = model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
#     class_weight=class_weight,
    callbacks=callback_GCN
#     callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

Epoch 1/1000


/Users/nikitaacharya/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:437: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("Adam/gradients/concat_1:0", shape=(None,), dtype=int64), values=Tensor("Adam/gradients/concat:0", shape=(None,), dtype=float32), dense_shape=Tensor("gradient_tape/masked_loss_and_accuracy/Cast_2:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)


10/10 [==============================] - 6s 382ms/step - loss: 1.2191 - acc: 0.5672 - precision_11: 0.1774 - recall_11: 0.6839 - val_loss: 0.8907 - val_acc: 0.1551 - val_precision_11: 0.1412 - val_recall_11: 1.0000
Epoch 2/1000
10/10 [==============================] - 2s 141ms/step - loss: 1.2175 - acc: 0.2182 - precision_11: 0.1154 - recall_11: 0.9568 - val_loss: 0.9088 - val_acc: 0.1551 - val_precision_11: 0.1412 - val_recall_11: 1.0000
Epoch 3/1000
10/10 [==============================] - 2s 176ms/step - loss: 0.8427 - acc: 0.3284 - precision_11: 0.1357 - recall_11: 0.9802 - val_loss: 0.9139 - val_acc: 0.1551 - val_precision_11: 0.1412 - val_recall_11: 1.0000
Epoch 4/1000
10/10 [==============================] - 2s 139ms/step - loss: 0.9557 - acc: 0.1566 - precision_11: 0.1101 - recall_11: 0.9942 - val_loss: 0.8855 - val_acc: 0.1551 - val_precision_11: 0.1412 - val_recall_11: 1.0000
Epoch 5/1000
10/10 [==============================] - 2s 172ms/step - loss: 0.8685 - acc: 0.2109 - pr

Epoch 37/1000
10/10 [==============================] - 2s 247ms/step - loss: 0.6886 - acc: 0.4520 - precision_11: 0.1633 - recall_11: 0.9840 - val_loss: 0.8114 - val_acc: 0.5255 - val_precision_11: 0.2179 - val_recall_11: 0.9333
Epoch 38/1000
10/10 [==============================] - 2s 245ms/step - loss: 0.6078 - acc: 0.5057 - precision_11: 0.1950 - recall_11: 0.9831 - val_loss: 0.8285 - val_acc: 0.5463 - val_precision_11: 0.2258 - val_recall_11: 0.9333
Epoch 39/1000
10/10 [==============================] - 2s 253ms/step - loss: 0.5626 - acc: 0.4944 - precision_11: 0.2032 - recall_11: 0.9937 - val_loss: 0.7166 - val_acc: 0.6667 - val_precision_11: 0.2812 - val_recall_11: 0.9000
Epoch 40/1000
10/10 [==============================] - 2s 276ms/step - loss: 0.5460 - acc: 0.5397 - precision_11: 0.2065 - recall_11: 0.9871 - val_loss: 1.1260 - val_acc: 0.6991 - val_precision_11: 0.2222 - val_recall_11: 0.4667
Epoch 41/1000
10/10 [==============================] - 2s 233ms/step - loss: 0.5531 

Epoch 73/1000
10/10 [==============================] - 2s 244ms/step - loss: 0.4135 - acc: 0.7528 - precision_11: 0.3321 - recall_11: 0.9678 - val_loss: 4.2980 - val_acc: 0.8125 - val_precision_11: 0.3158 - val_recall_11: 0.3000
Epoch 74/1000
10/10 [==============================] - 2s 252ms/step - loss: 0.5135 - acc: 0.6519 - precision_11: 0.2815 - recall_11: 0.9744 - val_loss: 4.2332 - val_acc: 0.8449 - val_precision_11: 0.4533 - val_recall_11: 0.5667
Epoch 75/1000
10/10 [==============================] - 2s 130ms/step - loss: 0.5893 - acc: 0.5715 - precision_11: 0.2009 - recall_11: 0.9766 - val_loss: 4.5182 - val_acc: 0.8079 - val_precision_11: 0.3736 - val_recall_11: 0.5667
Epoch 76/1000
10/10 [==============================] - 2s 236ms/step - loss: 0.4172 - acc: 0.7334 - precision_11: 0.3142 - recall_11: 0.9776 - val_loss: 2.3280 - val_acc: 0.4931 - val_precision_11: 0.1302 - val_recall_11: 0.4667
Epoch 77/1000
10/10 [==============================] - 2s 253ms/step - loss: 0.5446 

...

10/10 [==============================] - 2s 226ms/step - loss: 0.3545 - acc: 0.7617 - precision_11: 0.3457 - recall_11: 0.9918 - val_loss: 10.8154 - val_acc: 0.8356 - val_precision_11: 0.3103 - val_recall_11: 0.1500
Epoch 842/1000
10/10 [==============================] - 2s 223ms/step - loss: 0.1899 - acc: 0.8705 - precision_11: 0.5044 - recall_11: 0.9952 - val_loss: 9.6066 - val_acc: 0.7917 - val_precision_11: 0.1875 - val_recall_11: 0.1500
Epoch 843/1000
10/10 [==============================] - 2s 236ms/step - loss: 0.2136 - acc: 0.8463 - precision_11: 0.4871 - recall_11: 0.9969 - val_loss: 10.4018 - val_acc: 0.8565 - val_precision_11: 0.4500 - val_recall_11: 0.1500
Epoch 844/1000
10/10 [==============================] - 2s 224ms/step - loss: 0.2669 - acc: 0.8174 - precision_11: 0.4296 - recall_11: 0.9945 - val_loss: 11.4258 - val_acc: 0.8588 - val_precision_11: 0.0000e+00 - val_recall_11: 0.0000e+00
Epoch 845/1000
10/10 [==============================] - 2s 240ms/step - loss: 0.3375

10/10 [==============================] - 2s 233ms/step - loss: 0.2226 - acc: 0.8453 - precision_11: 0.4310 - recall_11: 0.9891 - val_loss: 8.5918 - val_acc: 0.8009 - val_precision_11: 0.3289 - val_recall_11: 0.4167
Epoch 877/1000
10/10 [==============================] - 2s 245ms/step - loss: 0.2901 - acc: 0.7795 - precision_11: 0.3519 - recall_11: 0.9963 - val_loss: 8.1783 - val_acc: 0.8009 - val_precision_11: 0.3289 - val_recall_11: 0.4167
Epoch 878/1000
10/10 [==============================] - 2s 236ms/step - loss: 0.3498 - acc: 0.7733 - precision_11: 0.3419 - recall_11: 0.9912 - val_loss: 9.4380 - val_acc: 0.7801 - val_precision_11: 0.2941 - val_recall_11: 0.4167
Epoch 879/1000
10/10 [==============================] - 2s 238ms/step - loss: 0.3242 - acc: 0.7554 - precision_11: 0.3518 - recall_11: 0.9985 - val_loss: 9.5561 - val_acc: 0.7685 - val_precision_11: 0.1552 - val_recall_11: 0.1500
Epoch 880/1000
10/10 [==============================] - 2s 226ms/step - loss: 0.3458 - acc: 0.7

Epoch 911/1000
10/10 [==============================] - 2s 236ms/step - loss: 0.1263 - acc: 0.9099 - precision_11: 0.6082 - recall_11: 0.9967 - val_loss: 16.2211 - val_acc: 0.8588 - val_precision_11: 0.0000e+00 - val_recall_11: 0.0000e+00
Epoch 912/1000
10/10 [==============================] - 2s 230ms/step - loss: 0.2883 - acc: 0.7946 - precision_11: 0.3792 - recall_11: 0.9978 - val_loss: 13.6080 - val_acc: 0.8912 - val_precision_11: 0.6757 - val_recall_11: 0.4167
Epoch 913/1000
10/10 [==============================] - 2s 226ms/step - loss: 0.1749 - acc: 0.8952 - precision_11: 0.5847 - recall_11: 0.9897 - val_loss: 15.2927 - val_acc: 0.9097 - val_precision_11: 0.8621 - val_recall_11: 0.4167
Epoch 914/1000
10/10 [==============================] - 2s 126ms/step - loss: 0.3441 - acc: 0.7551 - precision_11: 0.3032 - recall_11: 0.9931 - val_loss: 14.5720 - val_acc: 0.8565 - val_precision_11: 0.0000e+00 - val_recall_11: 0.0000e+00
Epoch 915/1000
10/10 [==============================] - 2s 1

Epoch 946/1000
10/10 [==============================] - 2s 227ms/step - loss: 0.2414 - acc: 0.8140 - precision_11: 0.3829 - recall_11: 0.9960 - val_loss: 12.0236 - val_acc: 0.8426 - val_precision_11: 0.0000e+00 - val_recall_11: 0.0000e+00
Epoch 947/1000
10/10 [==============================] - 2s 237ms/step - loss: 0.1959 - acc: 0.8685 - precision_11: 0.4832 - recall_11: 0.9880 - val_loss: 10.9559 - val_acc: 0.8889 - val_precision_11: 0.8000 - val_recall_11: 0.2667
Epoch 948/1000
10/10 [==============================] - 2s 239ms/step - loss: 0.1661 - acc: 0.8786 - precision_11: 0.5255 - recall_11: 0.9951 - val_loss: 11.8500 - val_acc: 0.8634 - val_precision_11: 0.5102 - val_recall_11: 0.4167
Epoch 949/1000
10/10 [==============================] - 2s 226ms/step - loss: 0.1456 - acc: 0.8921 - precision_11: 0.5519 - recall_11: 0.9971 - val_loss: 11.6624 - val_acc: 0.8866 - val_precision_11: 0.6410 - val_recall_11: 0.4167
Epoch 950/1000
10/10 [==============================] - 2s 226ms/ste

10/10 [==============================] - 2s 236ms/step - loss: 0.1923 - acc: 0.8686 - precision_11: 0.5015 - recall_11: 0.9928 - val_loss: 11.4263 - val_acc: 0.8356 - val_precision_11: 0.0000e+00 - val_recall_11: 0.0000e+00
Epoch 982/1000
10/10 [==============================] - 2s 161ms/step - loss: 0.2567 - acc: 0.7987 - precision_11: 0.3540 - recall_11: 0.9963 - val_loss: 10.9962 - val_acc: 0.8356 - val_precision_11: 0.3103 - val_recall_11: 0.1500
Epoch 983/1000
10/10 [==============================] - 2s 242ms/step - loss: 0.1548 - acc: 0.8952 - precision_11: 0.5767 - recall_11: 0.9925 - val_loss: 11.4297 - val_acc: 0.8310 - val_precision_11: 0.2903 - val_recall_11: 0.1500
Epoch 984/1000
10/10 [==============================] - 2s 251ms/step - loss: 0.2708 - acc: 0.8022 - precision_11: 0.3912 - recall_11: 0.9967 - val_loss: 12.0264 - val_acc: 0.8519 - val_precision_11: 0.4091 - val_recall_11: 0.1500
Epoch 985/1000
10/10 [==============================] - 2s 237ms/step - loss: 0.213

In [61]:
%tensorboard --logdir ./DetectFaultInCircuit12

Reusing TensorBoard on port 6009 (pid 76526), started 0:42:38 ago. (Use '!kill 76526' to kill it.)

In [63]:
model_loss = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done. Test loss: {}".format(model_loss))

2/2 [==============================] - 0s 15ms/step - loss: 5.3755 - acc: 0.6744 - precision_11: 0.1258 - recall_11: 0.2176
Done. Test loss: [5.3754658699035645, 0.6744415163993835, 0.12583668529987335, 0.21759259700775146]


In [64]:
# Evaluate model
from sklearn.metrics import classification_report

X_te = dataset[7].x
A_te = dataset[7].a
y_te = dataset[7].y

y_pred = model.predict([X_te, A_te, A_te], batch_size=X_te.shape[0])
report = classification_report(y_te, (y_pred >= 0.5).astype(int))
print('GCN Classification Report: \n {}'.format(report))

GCN Classification Report: 
               precision    recall  f1-score   support

         0.0       0.94      0.99      0.96      3134
         1.0       0.80      0.48      0.60       406

    accuracy                           0.93      3540
   macro avg       0.87      0.73      0.78      3540
weighted avg       0.92      0.93      0.92      3540



In [65]:
from sklearn.metrics import classification_report

X_te = dataset[0].x
A_te = dataset[0].a
y_te = dataset[0].y

y_pred = model.predict([X_te, A_te, A_te], batch_size=X_te.shape[0])
report = classification_report(y_te, (y_pred >= 0.5).astype(int))
print('GCN Classification Report: \n {}'.format(report))

GCN Classification Report: 
               precision    recall  f1-score   support

         0.0       0.89      0.93      0.91      5442
         1.0       0.38      0.27      0.31       846

    accuracy                           0.84      6288
   macro avg       0.63      0.60      0.61      6288
weighted avg       0.82      0.84      0.83      6288



In [67]:
from sklearn.metrics import classification_report

X_te = dataset[10].x
A_te = dataset[10].a
y_te = dataset[10].y

y_pred = model.predict([X_te, A_te, A_te], batch_size=X_te.shape[0])
report = classification_report(y_te, (y_pred >= 0.5).astype(int))
print('GCN Classification Report: \n {}'.format(report))

GCN Classification Report: 
               precision    recall  f1-score   support

         0.0       0.84      0.65      0.74      1846
         1.0       0.12      0.28      0.17       318

    accuracy                           0.60      2164
   macro avg       0.48      0.47      0.45      2164
weighted avg       0.74      0.60      0.65      2164

